In [2]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
import pandas as pd
import os

# Set the path to the file you'd like to load
file_path = ""

# Load the latest version
# The actual file name might be different, adjust if needed\n",
path = kagglehub.dataset_download("lakshmi25npathi/online-retail-dataset")
files = os.listdir(path)
print(files)
print("Path to dataset files:", path)

100%|██████████| 43.3M/43.3M [00:00<00:00, 210MB/s]

Extracting files...


['online_retail_II.xlsx']
Path to dataset files: /root/.cache/kagglehub/datasets/lakshmi25npathi/online-retail-dataset/versions/1


In [3]:
df = pd.read_excel(os.path.join(path, "online_retail_II.xlsx"))

print(df.head())

#df.columns.to_list()

#print(df.info())

  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom  


# Task 1: Data Cleaning

In [4]:
# Handle missing values in CustomerID
df.dropna(subset=['Customer ID'], inplace=True)

# Handle missing values in Description (can be filled or dropped, dropping for simplicity here)
df.dropna(subset=['Description'], inplace=True)

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Handle canceled orders (where InvoiceNo starts with 'C')
df = df[~df['Invoice'].astype(str).str.startswith('C')]

# Remove or cap outliers in Quantity and UnitPrice
# Removing rows with negative Quantity or UnitPrice
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]

# Convert InvoiceDate to proper datetime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

print("Data Cleaning Complete.")
print("Cleaned DataFrame Info:")
print(df.info())

Data Cleaning Complete.
Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 400916 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      400916 non-null  object        
 1   StockCode    400916 non-null  object        
 2   Description  400916 non-null  object        
 3   Quantity     400916 non-null  int64         
 4   InvoiceDate  400916 non-null  datetime64[ns]
 5   Price        400916 non-null  float64       
 6   Customer ID  400916 non-null  float64       
 7   Country      400916 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.5+ MB
None


/tmp/ipython-input-4-4292840022.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])


# Task 2: Exploratory Data Analysis (EDA)

### --- General Overview ---

In [5]:
# How many unique products are there?
print(f"Number of unique products: {df['Description'].nunique()}")

# What is the total number of transactions?
print(f"Total number of transactions: {df['Invoice'].nunique()}")

# How many unique customers are there?
print(f"Number of unique customers: {df['Customer ID'].nunique()}")

# What countries do the customers belong to?
print(f"Unique countries: {df['Country'].unique().tolist()}")
print(f"Number of unique countries: {df['Country'].nunique()}")

Number of unique products: 4444
Total number of transactions: 19213
Number of unique customers: 4312
Unique countries: ['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE', 'Germany', 'Portugal', 'Denmark', 'Netherlands', 'Poland', 'Channel Islands', 'Spain', 'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden', 'United Arab Emirates', 'Finland', 'Italy', 'Switzerland', 'Japan', 'Unspecified', 'Nigeria', 'Malta', 'RSA', 'Singapore', 'Bahrain', 'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Korea', 'Brazil', 'Canada', 'Iceland']
Number of unique countries: 37


### --- Product Analysis ---

In [6]:
# Calculate total revenue for each row
df['TotalPrice'] = df['Quantity'] * df['Price']

# What are the top 10 selling products by quantity?
top_10_selling_products_quantity = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 selling products by Quantity:")
print(top_10_selling_products_quantity)

# What are the top 10 revenue-generating products?
top_10_selling_products_revenue = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 revenue-generating products:")
print(top_10_selling_products_revenue)

# Which products have negative or zero unit prices? (Already handled in cleaning, but verifying)
products_zero_price = df[df['Price'] == 0]['Description'].unique()
print(f"\nProducts with zero unit price (should be empty after cleaning): {products_zero_price}")


Top 10 selling products by Quantity:
Description
WHITE HANGING HEART T-LIGHT HOLDER    56814
WORLD WAR 2 GLIDERS ASSTD DESIGNS     54610
BROCADE RING PURSE                    48166
PACK OF 72 RETRO SPOT CAKE CASES      45129
ASSORTED COLOUR BIRD ORNAMENT         44431
60 TEATIME FAIRY CAKE CASES           35656
PACK OF 60 PINK PAISLEY CAKE CASES    30871
JUMBO BAG RED RETROSPOT               29519
SMALL POPCORN HOLDER                  25703
BLACK AND WHITE PAISLEY FLOWER MUG    25685
Name: Quantity, dtype: int64

Top 10 revenue-generating products:
Description
WHITE HANGING HEART T-LIGHT HOLDER     151339.16
REGENCY CAKESTAND 3 TIER               143727.60
Manual                                  98531.99
ASSORTED COLOUR BIRD ORNAMENT           70291.03
JUMBO BAG RED RETROSPOT                 51644.25
POSTAGE                                 48741.08
ROTATING SILVER ANGELS T-LIGHT HLDR     40156.05
PAPER CHAIN KIT 50'S CHRISTMAS          36871.55
PARTY BUNTING                           

### --- Customer Analysis ---

In [7]:
# Which customers bought the most products? (by quantity)
top_customers_quantity = df.groupby('Customer ID')['Quantity'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 customers by total quantity purchased:")
print(top_customers_quantity)

# What is the distribution of purchases per customer? (by number of invoices)
purchases_per_customer = df.groupby('Customer ID')['Invoice'].nunique()
print("\nDistribution of purchases per customer (by number of unique invoices):")
print(purchases_per_customer.describe())

# Are there any loyal customers? (Customers with many purchases or high total spending)
# We can look at top customers by frequency (number of invoices) and monetary value
top_customers_frequency = df.groupby('Customer ID')['Invoice'].nunique().sort_values(ascending=False).head(10)
print("\nTop 10 most frequent customers (by number of invoices):")
print(top_customers_frequency)

top_customers_monetary = df.groupby('Customer ID')['TotalPrice'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 most valuable customers (by total spending):")
print(top_customers_monetary)


Top 10 customers by total quantity purchased:
Customer ID
13902.0    220600
14646.0    170278
13694.0    125893
18102.0    124216
14156.0    108105
14277.0     87830
13687.0     87167
17940.0     75825
14911.0     69709
16754.0     63551
Name: Quantity, dtype: int64

Distribution of purchases per customer (by number of unique invoices):
count    4312.000000
mean        4.455705
std         8.170213
min         1.000000
25%         1.000000
50%         2.000000
75%         5.000000
max       205.000000
Name: Invoice, dtype: float64

Top 10 most frequent customers (by number of invoices):
Customer ID
14911.0    205
17850.0    155
12748.0    144
15311.0    121
13089.0    109
14606.0    102
14156.0    102
13694.0     94
17841.0     91
18102.0     89
Name: Invoice, dtype: int64

Top 10 most valuable customers (by total spending):
Customer ID
18102.0    349164.35
14646.0    248396.50
14156.0    196549.74
14911.0    152121.22
13694.0    131443.19
17511.0     84541.17
15061.0     83284.38
166

### --- Time Series & Seasonal Trends ---

In [8]:
# What are the monthly sales trends?
df['InvoiceMonth'] = df['InvoiceDate'].dt.to_period('M')
monthly_sales = df.groupby('InvoiceMonth')['TotalPrice'].sum()
print("\nMonthly Sales Trends:")
print(monthly_sales)

# What are the daily patterns of transactions?
df['InvoiceDayOfWeek'] = df['InvoiceDate'].dt.day_name()
daily_transactions = df.groupby('InvoiceDayOfWeek')['Invoice'].nunique().reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
print("\nDaily Transaction Patterns:")
print(daily_transactions)

# Identify peak sales months and slow seasons.
print(f"\nPeak sales month: {monthly_sales.idxmax()} with sales of {monthly_sales.max():,.2f}")
print(f"Slowest sales month: {monthly_sales.idxmin()} with sales of {monthly_sales.min():,.2f}")


Monthly Sales Trends:
InvoiceMonth
2009-12     683504.010
2010-01     555802.672
2010-02     504558.956
2010-03     696978.471
2010-04     591982.002
2010-05     597833.380
2010-06     636371.130
2010-07     589736.170
2010-08     602224.600
2010-09     829013.951
2010-10    1033112.010
2010-11    1166460.022
2010-12     310656.370
Freq: M, Name: TotalPrice, dtype: float64

Daily Transaction Patterns:
InvoiceDayOfWeek
Monday       2986
Tuesday      3514
Wednesday    3426
Thursday     3976
Friday       2615
Saturday       30
Sunday       2666
Name: Invoice, dtype: int64

Peak sales month: 2010-11 with sales of 1,166,460.02
Slowest sales month: 2010-12 with sales of 310,656.37


### --- Country Analysis ---

In [9]:
# Which countries have the highest number of orders? (by unique invoices)
orders_by_country = df.groupby('Country')['Invoice'].nunique().sort_values(ascending=False)
print("\nCountries with highest number of orders:")
print(orders_by_country.head(10)) # Display top 10 countries by order count

# Revenue comparison across countries
revenue_by_country = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)
print("\nRevenue comparison across countries:")
print(revenue_by_country.head(10)) # Display top 10 countries by revenue


Countries with highest number of orders:
Country
United Kingdom    17612
Germany             347
EIRE                316
France              236
Netherlands         135
Sweden               68
Spain                66
Belgium              52
Australia            40
Switzerland          40
Name: Invoice, dtype: int64

Revenue comparison across countries:
Country
United Kingdom    7381644.433
EIRE               356041.860
Netherlands        268784.350
Germany            202025.391
France             146107.070
Sweden              53147.990
Denmark             50906.850
Spain               47568.650
Switzerland         43921.390
Australia           31446.800
Name: TotalPrice, dtype: float64


### --- RFM (Recency, Frequency, Monetary) Segmentation ---

In [10]:
# Calculate Recency, Frequency, Monetary
# Recency: Days since last purchase
# Frequency: Number of unique invoices
# Monetary: Total spending

# Define a snapshot date (a day after the last transaction in the dataset)
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm_df = df.groupby('Customer ID').agg(
    Recency=('InvoiceDate', lambda date: (snapshot_date - date.max()).days),
    Frequency=('Invoice', 'nunique'),
    Monetary=('TotalPrice', 'sum')
).reset_index()

print("\nRFM DataFrame:")
print(rfm_df.head())


RFM DataFrame:
   Customer ID  Recency  Frequency  Monetary
0      12346.0      165         11    372.86
1      12347.0        3          2   1323.32
2      12348.0       74          1    222.16
3      12349.0       43          3   2671.14
4      12351.0       11          1    300.93


# **EDA complete. Visualizations and Power BI dashboard file name** = Project_2_Online_Retail_Sales_EDA.pbix

### Save the cleaned dataset for Dashboard

In [11]:
output_path = "cleaned_online_retail_II.csv"
df.to_csv(output_path, index=False)

print(f"\nCleaned data saved to: {output_path}")



Cleaned data saved to: cleaned_online_retail_II.csv
